# Multi-Class SVC 

### Introduction

In its most basic form SVC is a binary classifier. However, many binary classification are only a subset of the typical classification problems encountered in practice. It is easy to generalise the base SVC model into a multi-class classifier. There are two common approaches to this: OvO (One-vs-One) and OvR (One-vs-the-rest). Let us dicuss these methods from a high-level overview, 

- **OvR**: OvR is short for _"One-vs-the-rest"_. In this approach, a binary classifier is made for each class label. Let us say that we have $M$ of classes, in OvR we create $M$ number of binary classifiers for each class label. When training each classifier, we re-label our data points so that positive class $1$ belongs to our choosen class $k$ and our negative class $0$ includes data points of all other classes. Mathematically,

$$
y_i^{(k)} = \begin{cases} +1, & y_i = k, \\ 0, & y_i \neq k. \end{cases}
$$

And so each SVM is trained to classify $y_i = k$ from $y_i \neq k$

- **OvO**:

### OvR Implementation

In OvR, the final class prediction for a given sample $X_i$ is determined from the BaseSVC whose decision function $f_k(X_i)$ is the largest. Note that $k$ denotes the $k$-th class label. We can imagine $f_k(X_i) = X_iW^T_{(k)} + b_{(k)}$ as a measure of how strongly a sample belongs to the class $k$. Mathematically,

$$
\hat{y} = \text{argmax}_k \quad f_k(X_i)
$$

where $\hat{y}$ is our predicted class label.

In [57]:
from models.BaseSVM import BaseSVC
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
import numpy as np

class SVC():
    """This is the class for the support vector classifier with multi-class support using a OvR approach."""

    def __init__(self, C=1):
        """Constructor method."""

        # Hyperparameters,
        self.C = C

        # Class attributes,
        self.class_labels, self.n_labels = None, None
        self.model_score = None

        # Stores all base SVC models,
        self.base_clfs = []

    def fit(self, X, y, n_jobs=-1):
        """DocString to be filled out."""

        # Extracting class labels,
        self.class_labels = np.unique(y)
        self.n_labels = len(self.class_labels)

        # Fitting BaseSVCs in parallel,
        self.base_clfs = Parallel(n_jobs=n_jobs)(
            delayed(self._fit_baseSVC)(X, y, class_label) for class_label in self.class_labels
            )

    def predict(self, X):
        """DocString to be filled out."""

        # Creating prediction matrix,
        preds_matrix = np.zeros(shape=(self.n_labels, X.shape[0]))
        for i, clf in enumerate(self.base_clfs):
            preds = clf._decision_function(X)
            preds_matrix[i] = preds

        # Returning predictions,
        preds = np.zeros(shape=X.shape[0], dtype=int)

        for i, base_preds in enumerate(preds_matrix.T):
            label_pred = np.argmax(base_preds)
            preds[i] = label_pred
        
        return preds
    
    def score(self, X, y):
        """Returns the classification accuracy on the supplied dataset."""

        # Calculating model predictions,
        y_preds = self.predict(X)

        # Computing classification accuracy,
        self.model_score = np.mean(y_preds == y)

        return self.model_score

    def _fit_baseSVC(self, X, y, class_label):
        """Helper function for fitting a BaseSVC model."""

        # Finding class labels,
        class_idxs = np.where(y == class_label)[0]
        class_comp_idxs = np.where(y != class_label)[0] # <-- The compliment.

        # Re-labeling class labels,
        X_class, y_class = X[class_idxs], np.ones(shape=len(class_idxs))
        X_class_comp, y_class_comp = X[class_comp_idxs], np.zeros(shape=len(class_comp_idxs))

        # Combining and splitting,
        X_new = np.concatenate((X_class, X_class_comp), axis=0)
        y_new = np.concatenate((y_class, y_class_comp), axis=0)

        # Training BaseSVM model,
        clf = BaseSVC(C=self.C)
        clf.fit(X_new, y_new)

        return clf

### Basic Example

In [58]:
# Importing dataset,
from sklearn.datasets import load_wine

# Creating data split,
wine_dataset = load_wine()
X, y = wine_dataset["data"], wine_dataset["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Training model,
clf = SVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9722222222222222